In [ ]:
import numpy as np
import helper_func
from sklearn import linear_model
import cPickle as pickle
import models
from sklearn.metrics.pairwise import cosine_similarity as cosdist
import edward as ed
from edward.models import Poisson,Gamma
import tensorflow as tf

In [ ]:
# PIK = "../models/bibtex_hpf_50.dat"
# with open(PIK, "rb") as f:
#     u,_,vy= pickle.load(f)
# f.close()
# dataset = 'multi_bibtex'
# x_train,y_train,x_test,y_test = helper_func.load_data(dataset)
# k = u.shape[1]
# w = np.zeros(shape=[x_train.shape[1],u.shape[1]])

In [ ]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()
PIK = "../models/bibtex_po_po_50.dat"
with open(PIK, "rb") as f:
    [u,_,vy]= pickle.load(f)
f.close()
dataset = 'multi_bibtex'
x_train,y_train,x_test,y_test = helper_func.load_data(dataset)

users = u.shape[0]
k = u.shape[1]
items2 = vy.shape[1]


# k = u.shape[1]
w = np.zeros(shape=[x_train.shape[1],u.shape[1]])

In [ ]:
print items2

In [ ]:
temp = np.matmul(u,vy)
result = 0.0
for i in range(0,temp.shape[0]):
    result += helper_func.patk(temp[i],y_train[i],1)
result /= temp.shape[0]
print result

In [ ]:
for i in range(0,k):
    print i
    reg = linear_model.Ridge(alpha = 0.2)
    reg.fit(x_train,u[:,i])
    w[:,i] = reg.coef_
u_hat = np.matmul(x_train,w)
u_test = np.matmul(x_test,w)

In [ ]:
nb = 10
result = 0.0
for i in range(0,u_test.shape[0]):
    dist = cosdist(np.reshape(u_test[i],newshape=[1,-1]),u_hat)
    indices = np.argsort(dist)[:,:nb]
    predicted = np.zeros(shape=y_train.shape[1],dtype=np.float64)
    for j in range(0,nb):
        predicted += y_train[indices[0,j]]
    predicted /= nb
    result += helper_func.patk(predicted,y_test[i],1)
result /= u_test.shape[0]
print result